# Serving Runtime Remote Artifacts

In [ ]:
import mlrun
import os

# Initialize the MLRun project object
project = mlrun.load_project('./')

# Required credentials :
# AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, GOOGLE_APPLICATION_CREDENTIALS as jupyter env vars and project secrets (k8s provider).

In [ ]:
# Setting dummy serving function
project.set_function(mlrun.new_function(name='serving', kind='serving',image='mlrun/ml-models', command = []))
project.get_function('serving').spec.build.commands = ['pip install -U xgboost']

# adding our remote source model
model_artifact = project.get_artifact('log-transactions-log_transactions_bst_model')
model_path = model_artifact.get_target_path() + model_artifact.model_file

project.get_function('serving').add_model(key = 'my_model',
                                          class_name = "mlrun.frameworks.sklearn.SKLearnModelServer",
                                          model_path = model_path)

# mounting
project.get_function('serving').apply(mlrun.platforms.mount_s3())

In [ ]:
server = project.get_function('serving').to_mock_server()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

server.test(body={'inputs': X_test.tolist()[:3]})

In [ ]:
project.deploy_function('serving')

In [ ]:
project.get_function('serving').invoke('/v2/models/my_model/infer', body={'inputs': X_test.tolist()[:3]})

# Pipeline remote access

In [ ]:
%%writefile workflow.py
import mlrun
from kfp import dsl


@dsl.pipeline(name="Mask Detection Pipeline")
def kfpipeline():
    # Get our project object:
    project = mlrun.get_current_project()
    
    project.get_function('log_transactions').apply(mlrun.platforms.mount_s3())
    run = project.run_function('log_transactions')
    
    model_artifact = project.get_artifact('log-transactions-log_transactions_bst_model')
    model_path = model_artifact.get_target_path() + model_artifact.model_file
    
    project.get_function('serving').add_model(key = 'my_model',
                                          class_name = "mlrun.frameworks.sklearn.SKLearnModelServer",
                                          model_path = model_path)
    project.get_function('log_transactions').apply(mlrun.platforms.mount_s3())
    project.deploy_function('serving').after(run)
    return

In [ ]:
# Register the workflow file:
workflow_name = "iris_workflow"
project.set_workflow(workflow_name, "workflow.py")

# Save the project:
project.save()

In [ ]:
# Running with S3
project.run(name=workflow_name,watch=True, local=False, artifact_path=project.artifact_path)
project.save()

In [ ]:
# Running with GCS
project.run(name=workflow_name,watch=True, local=False, artifact_path='gs' + project.artifact_path[2:])